In [290]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage
from langchain_core.tools import tool, InjectedToolArg
from typing import Annotated
import requests
import os
from langchain_ollama.chat_models import ChatOllama

In [291]:
# os.environ["OPENAI_API_BASE"] = "https://openrouter.ai/api/v1"
# os.environ["OPENAI_API_KEY"] = "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"

# model = ChatOpenAI(model="deepseek/deepseek-chat-v3-0324:free")

In [292]:
# Initialize the model and message
model = ChatOllama(model="llama3.1")

In [293]:
# create tools
@tool
def get_conversion(base_currency : str, target_currency: str) -> float:
    '''
    This function fetches the currency conversion factor between a given base currency and a target currency
    '''

    url = f'https://v6.exchangerate-api.com/v6/fae573cd0eff3ed235f7de34/pair/{base_currency}/{target_currency}'
    response = requests.get(url)

    return response.json()

@tool
def convert(base_currency_value:float, conversion_rate:Annotated[float, InjectedToolArg]) -> float:
    '''
    given a currency conversion rate this function calculates the target currency value from a given base currency
    '''
    return base_currency_value * conversion_rate

In [294]:
get_conversion.invoke({'base_currency':'USD','target_currency':'INR'})

{'result': 'success',
 'documentation': 'https://www.exchangerate-api.com/docs',
 'terms_of_use': 'https://www.exchangerate-api.com/terms',
 'time_last_update_unix': 1755820801,
 'time_last_update_utc': 'Fri, 22 Aug 2025 00:00:01 +0000',
 'time_next_update_unix': 1755907201,
 'time_next_update_utc': 'Sat, 23 Aug 2025 00:00:01 +0000',
 'base_code': 'USD',
 'target_code': 'INR',
 'conversion_rate': 87.2812}

In [295]:
convert.invoke({'base_currency_value':10,'conversion_rate':87.071})

870.71

In [296]:
query = input('Type here your query regarding conversion between two cureencys')

In [297]:
message = [HumanMessage(query)]

In [298]:
message

[HumanMessage(content='convert 100 inr in usd', additional_kwargs={}, response_metadata={})]

In [299]:
model_with_tools = model.bind_tools(
    [get_conversion, convert],
)

In [300]:
ai_message = model_with_tools.invoke(message)

In [301]:
message.append(ai_message)
ai_message.tool_calls

[{'name': 'get_conversion',
  'args': {'base_currency': 'INR', 'target_currency': 'USD'},
  'id': '168a3256-59fd-4801-bab3-1736aaebdd83',
  'type': 'tool_call'},
 {'name': 'convert',
  'args': {'base_currency_value': '100'},
  'id': '63c481f6-3d3e-4a0b-8c28-4addf8e0b49a',
  'type': 'tool_call'}]

In [302]:
import json
for tool_call in ai_message.tool_calls:
    # execute the 1st tool and get the value of conversion rate
    if tool_call['name'] == 'get_conversion':
        tool_message1 = get_conversion.invoke(tool_call)
        print(tool_message1)
        # fetch this conversion rate
        converstion_rate = json.loads(tool_message1.content)['conversion_rate']
        message.append(tool_message1)
        print(converstion_rate)

    # execute the 2nd tool using the conversion rate from tool1
    if tool_call['name'] == 'convert':
        # fetch the current arg
        tool_call['args']['conversion_rate'] = converstion_rate
        tool_message2 = convert.invoke(tool_call)
        message.append(tool_message2)
        

content='{"result": "success", "documentation": "https://www.exchangerate-api.com/docs", "terms_of_use": "https://www.exchangerate-api.com/terms", "time_last_update_unix": 1755820801, "time_last_update_utc": "Fri, 22 Aug 2025 00:00:01 +0000", "time_next_update_unix": 1755907201, "time_next_update_utc": "Sat, 23 Aug 2025 00:00:01 +0000", "base_code": "INR", "target_code": "USD", "conversion_rate": 0.01146}' name='get_conversion' tool_call_id='168a3256-59fd-4801-bab3-1736aaebdd83'
0.01146


In [303]:
message

[HumanMessage(content='convert 100 inr in usd', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'llama3.1', 'created_at': '2025-08-22T06:07:22.405675034Z', 'done': True, 'done_reason': 'stop', 'total_duration': 14628855265, 'load_duration': 242426641, 'prompt_eval_count': 246, 'prompt_eval_duration': 2739730210, 'eval_count': 49, 'eval_duration': 11643954055, 'model_name': 'llama3.1'}, id='run--582a4601-2d70-4d45-a68b-b8846163bd5d-0', tool_calls=[{'name': 'get_conversion', 'args': {'base_currency': 'INR', 'target_currency': 'USD'}, 'id': '168a3256-59fd-4801-bab3-1736aaebdd83', 'type': 'tool_call'}, {'name': 'convert', 'args': {'base_currency_value': '100', 'conversion_rate': 0.01146}, 'id': '63c481f6-3d3e-4a0b-8c28-4addf8e0b49a', 'type': 'tool_call'}], usage_metadata={'input_tokens': 246, 'output_tokens': 49, 'total_tokens': 295}),
 ToolMessage(content='{"result": "success", "documentation": "https://www.exchangerate

In [304]:
model_with_tools.invoke(message).content

'The conversion of 100 INR to USD is approximately 1.15 USD.'